We will assume the following in this problem: 

You have 100 users, who use your main app, which resembles Spotify. 
You are launching a new feature for an app imitating “Discover weekly”. 
This new feature will also suggest to the users what songs they can listen to, that they may like.
This means that you will be looking at what songs people played in the app and we will suggest new songs to them,
in the new feature, based on what we know. The suggested 5 songs may be songs the user has played. 


Week 1 after the day of the launch - Individual choice
Each of your 100 users is trying the new Discover feature in your app for the first time, in the first week. The feature will generate songs that he or she may like. You need to generate 5 songs. Assume this is the first week in which you launched your app on the google store. 
 
Assume that all users start using the feature on day 1 and no one downloads it later. The 5 songs that will be generated for the first time we pick in the following way:  
 
You already have 100 pre-made playlists, with 50 songs in each. Your task is the following: if you find a playlist that contains 3 songs that the user has listened to already and 3 he has not heard yet, you select this playlist. From this playlist, your algorithm picks any 5 songs. 


Task
Write a Python computer program that can simulate Spotify’s “Discover Weekly” as mentioned above.

Minimum requirements

- Give the pseudocode of your program.

- Make a flowchart of your program.

- Put comments in your code.

- The program generates suggestions for the users of Spotify: a few new songs every few days.

 

In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv('spotify-dataset.csv')
df_playlists = pd.DataFrame() # Create a new dataframe to store the playlists
#df
playlists = {}
for i in range(0,100): # Loop through the first 100 playlists
    df2 = []
    for j in range(0,50): # Loop through the first 50 songs in each playlist
        df2.append(df.sample().values.tolist()[0]) # Get a random song from the dataset
    playlists[i] = df2
print(playlists)

In [3]:
#print(df.sample().values.tolist()[0])


df.columns = [c.replace(' ', '_') for c in df.columns]
print(df)

                                                 title            artist  \
0                                     Hey, Soul Sister             Train   
1                                 Love The Way You Lie            Eminem   
2                                              TiK ToK             Kesha   
3                                          Bad Romance         Lady Gaga   
4                                 Just the Way You Are        Bruno Mars   
..                                                 ...               ...   
598                Find U Again (feat. Camila Cabello)       Mark Ronson   
599      Cross Me (feat. Chance the Rapper & PnB Rock)        Ed Sheeran   
600  No Brainer (feat. Justin Bieber, Chance the Ra...         DJ Khaled   
601    Nothing Breaks Like a Heart (feat. Miley Cyrus)       Mark Ronson   
602                                   Kills You Slowly  The Chainsmokers   

    the_genre_of_the_track  year  Beats.Per.Minute_-The_tempo_of_the_song  \
0         

In [4]:
import random


class User: 
    liked_songs = []
    categories = []
    def __init__(self) -> None:
        self.liked_songs = []
        self.categories = []
        
    def add_random_song(self):
        self.liked_songs.append(df.sample().values.tolist()[0]) # Get a random song from the dataset
    
    def find_playlist(self,playlists):
        common_songs =[]
        for i in playlists:
            playlist = (playlists[i]) 
            #print(playlist)
            playlistUpdated = []
            for song in playlist:
               # print(song[0])
                playlistUpdated.append(song[0])
            playlist = playlistUpdated
            #print(playlist)
            counter_exists =0 # counter for the same songs in both playlists
            counter_no_exist = 0 # counter for the songs that are not in the playlist

            for song in self.liked_songs: # Loop through the songs the user listened to
                if song[0] in playlist:
                    counter_exists+=1
                    common_songs.append(song)
                else:
                    counter_no_exist+=1 
            if counter_exists >2 and counter_no_exist >2: 
                return playlist,common_songs
    
    def determine_category(self):
        
        category =[] #list because there can possibly be two categories or more
        genres ={}
        for songs in self.liked_songs:
            #print(songs[2])
            genre = songs[2]
            if 'pop' in genre:
                genres['pop'] = genres.get('pop',0) + 1
            elif 'rock' in genre:
                genres['rock'] = genres.get('rock',0) + 1
            elif 'techno' in genre:
                genres['techno'] = genres.get('techno',0) + 1
        maximum = 0
        max2 = 0
        max_category = ""
        max2cat = ""
        for i in genres:
            if genres[i] >maximum:
                maximum =genres[i]
                max_category = i
            if genres[i]<maximum and genres[i]>max2:
                max2 = genres[i]
                max2cat = i
        
        if max2 != 0:
            if int((maximum/max2)*10) in range(9,10):
                return [max_category,max2cat]
            #print(max_category)
        return [max_category]


            
                
        

user1 = User()
for i in range(0,50):
    user1.add_random_song() # Add 50 random songs to the user's listened songs



selected_playlist = user1.find_playlist(playlists)[0]

selected_songs =random.sample(selected_playlist,5)

print(selected_songs)


["Runnin' (Lose It All)", 'Mi Gente (feat. Beyonc?', "Hold On, We're Going Home", 'Love Runs Out', 'I Like It']


Week 2 - Genres
In the second week you will use a different approach: 
A user opens the app and is looking for new suggestions on “Discover.”  
 
There are three types of music only: pop, rock, and techno.  
 
You want to find which style the user has listened to the most, of those three. Define in your own terms what this should mean. For example, you can say that if the user has listened to 20 rock songs and 2 techno songs, and 3 pop songs, this is a rock music fan. Or you can work with percentages: you can find a user that played 80% techno and 20% pop. You can make your algorithm disregard a difference of 10% and say that if a user played pop 45 percent of the time and rock 50 percent of the time, the difference is small and both styles should be considered equally important to him or her. Outline your scale and assumptions in your work. 

In the second week, your algorithm should suggest 5 songs again, this time based on the above-mentioned, i.e. based on style. 
 
It is possible that the program suggests one song in one week and again the same song in the following week.

In [5]:
user2 = User()
for i in range(0,50):
    user2.add_random_song() # Add 50 random songs to the user's listened songs

#print(user2.liked_songs) # Sample category from song

user2.determine_category()
df3 = pd.DataFrame()


def rec_song(): #recoommends songs
    df3 = df[df.the_genre_of_the_track == user2.determine_category()[0]].sample(5)
    return df3
rec_song()



,title,artist,the_genre_of_the_track,year,Beats.Per.Minute_-The_tempo_of_the_song,"Energy-_The_energy_of_a_song_-_the_higher_the_value,_the_more_energtic","Danceability_-_The_higher_the_value,_the_easier_it_is_to_dance_to_this_song","Loudness/dB_-_The_higher_the_value,_the_louder_the_song","Liveness_-_The_higher_the_value,_the_more_likely_the_song_is_a_live_recording","Valence_-_The_higher_the_value,_the_more_positive_mood_for_the_song",Length_-_The_duration_of_the_song,Acousticness_-_The_higher_the_value_the_more_acoustic_the_song_is,Speechiness_-_The_higher_the_value_the_more_spoken_word_the_song_contains,Popularity-_The_higher_the_value_the_more_popular_the_song_is
537,Look What You Made Me Do,Taylor Swift,pop,2018,128,71,77,-6,13,51,212,20,12,75
218,Maps,Maroon 5,pop,2014,120,71,74,-6,6,88,190,2,3,78
526,What Lovers Do (feat. SZA),Maroon 5,pop,2018,110,60,80,-5,10,42,200,8,6,79
137,It Will Rain,Bruno Mars,pop,2012,150,84,58,-7,8,48,258,34,5,38
109,I Knew You Were Trouble.,Taylor Swift,pop,2012,77,47,62,-7,3,68,220,0,4,77


Week 3 - Shifts
In the third week after the launch, you want to account for a user’s mood shift. We have a few types of songs: “happy”, “party”, “calming”, and “lounge.” Define in your own algorithm if you think one song can be classified as two of those at the same time.  
 
This week you need to select 5 more songs to suggest to each user, again. If last week the user was playing a lot of songs from one of those types, this week we want to suggest more of the same type. Define, in your own terms what this should mean: if the user listened to 3 “calming” songs, should we provide 3 more without considering other factors? What if he or she also listened to 3 “party” songs and 4 “lounge” style songs?

In [6]:
#happy = 1
#party = 2
#calming =3
#lounge = 4


import array
import time
import numpy
import random
df_train = pd.DataFrame()

#print(random.sample(range(0,len(df)), 50))

training_set_inputs = []
training_set_outputs = [[]]
for i in random.sample(range(0,len(df)), 10):
    training_set_inputs.append(df.loc[i, :].values.tolist()[-9:])
    print(df.loc[i, :].values.tolist())
    time.sleep(1)
    training_set_outputs[0].append(input("Please enter a category: #happy = 1 #party = 2 #calming =3# lounge = 4"))
#print(training_set_inputs)
tr_in = training_set_inputs
tr_out = training_set_outputs[0]
df_train['X'] = training_set_inputs
df_train['Y'] = training_set_outputs[0]
training_set_outputs = numpy.array(training_set_outputs).T
training_set_inputs = numpy.array(training_set_inputs)


sample_test = df.loc[555, :].values.tolist()[-9:]
print("Sample Test")
print(sample_test)

df_train

['Sugar (feat. Francesco Yates)', 'Robin Schulz', 'dance pop', 2015, 123, 82, 64, -5, 16, 64, 219, 2, 6, 60]
['Tell Me You Love Me - NOTD Remix', 'Demi Lovato', 'dance pop', 2018, 102, 77, 58, -5, 9, 76, 182, 5, 8, 44]
['Papi', 'Jennifer Lopez', 'dance pop', 2011, 120, 81, 71, -5, 37, 89, 223, 13, 6, 58]


KeyboardInterrupt: Interrupted by user

In [ ]:
from numpy import arange, average, exp, array, random, dot

class NeuralNetwork():
    def __init__(self):
        random.seed(1)
        self.synaptic_weights = 2 * random.random((9, 1)) - 1


    def __sigmoid(self, x):
        return 1 / (1 + exp(-x))


    def __sigmoid_derivative(self, x):
        return x * (1 - x)


    def train(self, training_set_inputs, training_set_outputs, number_of_training_iterations):
        for iteration in range(number_of_training_iterations):
            output = self.think(training_set_inputs)
            error = training_set_outputs - output
            adjustment = dot(training_set_inputs.T, error * self.__sigmoid_derivative(output))
            self.synaptic_weights += adjustment

  
    def think(self, inputs):
        return self.__sigmoid(dot(inputs, self.synaptic_weights))


In [ ]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
neural_network = NeuralNetwork()
model.fit(tr_in, tr_out)
#sample_test = df.loc[555, :].values.tolist()[:]
print(sample_test)
sample_test = df.loc[555, :].values.tolist()[4:12]
print(sample_test)
#print(model.predict([sample_test]))
#print ((neural_network.think(numpy.array([sample_test]))))

['Wait', 'Maroon 5', 'pop', 2018, 126, 60, 66, -5, 11, 45, 191, 10, 6, 67, [[66, -5, 11, 45, 191, 10, 6, 67]]]
[126, 60, 66, -5, 11, 45, 191, 10]


In [ ]:
from numpy import ndarray
from collections import Counter

user3 = User()
for i in range(0,50):
    user3.add_random_song()
df_nueral = pd.DataFrame(user3.liked_songs)

def predict_categories(liked_songs):
    training_set_a = []
    for song in liked_songs:
        training_set_a.append(song[-9:])
    #print(training_set_a)
    predicted_categories = []
    for song in training_set_a:
        #predicted_categories.append(ndarray.tolist((neural_network.think(numpy.array([song])[0])))[0])
        predicted_categories.append(ndarray.tolist(model.predict([song]))[0])
    #print(predicted_categories)

    return predicted_categories



for i in range(0,len(user3.liked_songs)):
    #print(str(user3.liked_songs[i][0]) + " " + str(user3.liked_songs[i][1]) + " ---------- Predicted Category: " + str(predict_categories(user3.liked_songs)[i]))
    user3.categories.append((predict_categories(user3.liked_songs)[i]))
#print(user3.categories)
Counter = Counter(user3.categories)
chosen_category = [max(set(user3.categories), key = user3.categories.count)]
most_occur = Counter.most_common(4)
print(most_occur)
if(len(most_occur)>1):
    print(100-(most_occur[1][1] / most_occur[0][1]) * 100)
    if 100-(most_occur[1][1] / most_occur[0][1]) * 100 <10:
        chosen_category = [most_occur[0][0] , most_occur[1][0]]
print(chosen_category)



[('2', 49), ('1', 1)]
97.95918367346938
['2']


In [ ]:
print(df.columns[-9:])



Index(['Energy-_The_energy_of_a_song_-_the_higher_the_value,_the_more_energtic',
       'Danceability_-_The_higher_the_value,_the_easier_it_is_to_dance_to_this_song',
       'Loudness/dB_-_The_higher_the_value,_the_louder_the_song',
       'Liveness_-_The_higher_the_value,_the_more_likely_the_song_is_a_live_recording',
       'Valence_-_The_higher_the_value,_the_more_positive_mood_for_the_song',
       'Length_-_The_duration_of_the_song',
       'Acousticness_-_The_higher_the_value_the_more_acoustic_the_song_is',
       'Speechiness_-_The_higher_the_value_the_more_spoken_word_the_song_contains',
       'Popularity-_The_higher_the_value_the_more_popular_the_song_is'],
      dtype='object')


In [ ]:
import numpy as np

temp_cols = df.columns[4:13]
#print(len(temp_cols))
df["Raw Text"] = ""

for cols in temp_cols:
    #print(df[cols][0])
    df["Raw Text"] +=  df[cols].astype(str) +"," 
#print(df["Raw Text"][0])
df["Raw Text"] = df["Raw Text"].apply(lambda x: x.split(","))
df["Raw Text"] = df["Raw Text"].apply(lambda x: x[0:9])
df["Raw Text"] = df["Raw Text"].apply(lambda x: [int(y) for y in x if y !=''])
#print(df["Raw Text"][0])
df["Raw Text"] = df["Raw Text"].apply(lambda x: [int(y) for y in x])
df["Raw Text"] = df["Raw Text"].apply(lambda x: [x])
#df["Raw Text"][0]

df['Predicted Category'] = df["Raw Text"].apply(model.predict)
#df

In [ ]:
df



,title,artist,the_genre_of_the_track,year,Beats.Per.Minute_-The_tempo_of_the_song,"Energy-_The_energy_of_a_song_-_the_higher_the_value,_the_more_energtic","Danceability_-_The_higher_the_value,_the_easier_it_is_to_dance_to_this_song","Loudness/dB_-_The_higher_the_value,_the_louder_the_song","Liveness_-_The_higher_the_value,_the_more_likely_the_song_is_a_live_recording","Valence_-_The_higher_the_value,_the_more_positive_mood_for_the_song",Length_-_The_duration_of_the_song,Acousticness_-_The_higher_the_value_the_more_acoustic_the_song_is,Speechiness_-_The_higher_the_value_the_more_spoken_word_the_song_contains,Popularity-_The_higher_the_value_the_more_popular_the_song_is,Raw Text,Predicted Category
0,"Hey, Soul Sister",Train,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83,"[[97, 89, 67, -4, 8, 80, 217, 19, 4]]",[2]
1,Love The Way You Lie,Eminem,detroit hip hop,2010,87,93,75,-5,52,64,263,24,23,82,"[[87, 93, 75, -5, 52, 64, 263, 24, 23]]",[2]
2,TiK ToK,Kesha,dance pop,2010,120,84,76,-3,29,71,200,10,14,80,"[[120, 84, 76, -3, 29, 71, 200, 10, 14]]",[2]
3,Bad Romance,Lady Gaga,dance pop,2010,119,92,70,-4,8,71,295,0,4,79,"[[119, 92, 70, -4, 8, 71, 295, 0, 4]]",[2]
4,Just the Way You Are,Bruno Mars,pop,2010,109,84,64,-5,9,43,221,2,4,78,"[[109, 84, 64, -5, 9, 43, 221, 2, 4]]",[2]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,Find U Again (feat. Camila Cabello),Mark Ronson,dance pop,2019,104,66,61,-7,20,16,176,1,3,75,"[[104, 66, 61, -7, 20, 16, 176, 1, 3]]",[2]
599,Cross Me (feat. Chance the Rapper & PnB Rock),Ed Sheeran,pop,2019,95,79,75,-6,7,61,206,21,12,75,"[[95, 79, 75, -6, 7, 61, 206, 21, 12]]",[2]
600,"No Brainer (feat. Justin Bieber, Chance the Ra...",DJ Khaled,dance pop,2019,136,76,53,-5,9,65,260,7,34,70,"[[136, 76, 53, -5, 9, 65, 260, 7, 34]]",[2]
601,Nothing Breaks Like a Heart (feat. Miley Cyrus),Mark Ronson,dance pop,2019,114,79,60,-6,42,24,217,1,7,69,"[[114, 79, 60, -6, 42, 24, 217, 1, 7]]",[2]
